the goal is to create a regression model where the model is fed a time series somehow

In [76]:
import graphlab as gl
import datetime
from datetime import timedelta


In [77]:
prices = gl.SFrame.read_csv('coinbaseUSD.csv',header=False)

Finished parsing file C:\Users\ADMIN\notebook\Bitcoin-Market-Regression\coinbaseUSD.csv

Parsing completed. Parsed 100 lines in 0.66978 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 1219153 lines. Lines per second: 1.11757e+006

Read 14592009 lines. Lines per second: 2.30857e+006

Finished parsing file C:\Users\ADMIN\notebook\Bitcoin-Market-Regression\coinbaseUSD.csv

Parsing completed. Parsed 27873479 lines in 10.922 secs.

In [78]:
prices.head()
prices['int_times']=prices['X1']

In [79]:
prices['X1']= prices['X1'].apply(lambda x: datetime.datetime.utcfromtimestamp(x) )

In [80]:
prices.rename({'X1':'date','X2':'price','X3':'volume'})

date,price,volume,int_times
2014-12-01 05:33:56,300.0,0.01,1417412036
2014-12-01 05:40:23,300.0,0.01,1417412423
2014-12-01 06:24:08,370.0,0.01,1417415048
2014-12-01 06:50:12,370.0,0.02655554,1417416612
2014-12-02 05:29:26,377.0,0.01,1417498166
2014-12-02 10:59:09,377.75,0.25,1417517949
2014-12-02 10:59:09,378.0,3.75,1417517949
2014-12-02 11:04:17,378.0,4.9,1417518257
2014-12-02 11:05:40,378.0,5.2,1417518340
2014-12-02 18:43:00,378.0,0.1,1417545780


In [81]:
prices['volume'].max()

897.982

In [82]:
time_price= gl.TimeSeries(prices,index='date')

In [83]:
delta_day = timedelta(days=1)
delta_hour =  timedelta(hours=2) 

In [84]:
daily_agg=time_price.resample(delta_day,downsample_method='mean',upsample_method='linear')
hourly_agg=time_price.resample(delta_hour,downsample_method='mean',upsample_method='linear')

In [85]:
roll_period=[1,2,3,4,5,10,20,30,40,50,100,200,300]




Creating derived columns for the daily data




In [86]:
for x in roll_period:
    temp_name = 'pri_rol_'+str(x)
    daily_agg[temp_name]=daily_agg['price'].rolling_mean(-1-x,-1,min_observations=0)

In [87]:
for x in roll_period:
    temp_name = 'vol_rol_'+str(x)
    daily_agg[temp_name]=daily_agg['volume'].rolling_sum(-1-x,-1,min_observations=0)

In [88]:
for x in roll_period:
    temp_name = 'min_rol_'+str(x)
    daily_agg[temp_name]=daily_agg['price'].rolling_min(-1-x,-1,min_observations=0)

In [89]:
for x in roll_period:
    temp_name = 'max_rol_'+str(x)
    daily_agg[temp_name]=daily_agg['price'].rolling_max(-1-x,-1,min_observations=0)




Creating the derived columns for the hourly Data




In [90]:
for x in roll_period:
    temp_name = 'pri_rol_'+str(x)
    hourly_agg[temp_name]=hourly_agg['price'].rolling_mean(-1-x,-1,min_observations=0)

In [91]:
for x in roll_period:
    temp_name = 'vol_rol_'+str(x)
    hourly_agg[temp_name]=hourly_agg['volume'].rolling_sum(-1-x,-1,min_observations=0)

In [92]:
model_data=daily_agg.to_sframe().remove_column('date').dropna()



In [131]:
model_data.head()

int_times,price,volume,pri_rol_1,pri_rol_2,pri_rol_3,pri_rol_4
1417526708.12,377.84375,1.8767,335.0,335.0,335.0,335.0
1417612312.0,377.818333333,0.0911014033333,356.421875,356.421875,356.421875,356.421875
1417657685.0,377.1,0.01,377.831041667,363.554027778,363.554027778,363.554027778
1417743885.0,377.55,0.0125,377.459166667,377.587361111,366.940520833,366.940520833
1417830085.0,378.0,0.015,377.325,377.489444444,377.578020833,369.062416667
1417953835.5,376.5025,0.06625,377.775,377.55,377.617083333,377.662416667
1418077586.0,375.005,0.1175,377.25125,377.350833333,377.288125,377.394166667
1418128481.25,377.1275,0.06375,375.75375,376.5025,376.764375,376.8315
1418179376.5,379.25,0.01,376.06625,376.211666667,376.65875,376.837
1418267479.45,371.074,0.32,378.18875,377.1275,376.97125,377.177


roll through our data updating our model at fixed intervals

In [120]:
prediction_list = []
for entry in  model_data:
    if entry['int_times']%200<2:
        prevmask = model_data['int_times']< entry['int_times']
        model = gl.regression.create(model_data[prevmask], target='price')
    price_prediction = model.predict(entry)
    prediction_list.append(price_prediction)

Boosted trees regression:

--------------------------------------------------------

Number of examples          : 77

Number of features          : 54

Number of unpacked features : 54

...

| 3         | 0.023061     | 1278.670288        | 689.119507           | 305.909027    | 210.332855      |

| 4         | 0.028074     | 1006.156616        | 416.605835           | 217.824554    | 144.745575      |

| 5         | 0.033087     | 820.313232         | 230.762451           | 156.093857    | 99.189224       |

| 6         | 0.039104     | 645.094482         | 159.159851           | 112.644844    | 69.864571       |

+-----------+--------------+--------------------+----------------------+---------------+-----------------+

In [129]:
print(prediction_list)

[dtype: float
Rows: 1
[341.7481689453125], dtype: float
Rows: 1
[345.66400146484375], dtype: float
Rows: 1
[367.92010498046875], dtype: float
Rows: 1
[367.92010498046875], dtype: float
Rows: 1
[367.92010498046875], dtype: float
Rows: 1
[367.92010498046875], dtype: float
Rows: 1
[367.92010498046875], dtype: float
Rows: 1
[366.5639343261719], dtype: float
Rows: 1
[367.92010498046875], dtype: float
Rows: 1
[367.92010498046875], dtype: float
Rows: 1
[366.3419189453125], dtype: float
Rows: 1
[364.9857482910156], dtype: float
Rows: 1
[342.7296447753906], dtype: float
Rows: 1
[342.7296447753906], dtype: float
Rows: 1
[335.6745910644531], dtype: float
Rows: 1
[335.6745910644531], dtype: float
Rows: 1
[335.6745910644531], dtype: float
Rows: 1
[335.6745910644531], dtype: float
Rows: 1
[330.8307800292969], dtype: float
Rows: 1
[330.8307800292969], dtype: float
Rows: 1
[330.8307800292969], dtype: float
Rows: 1
[330.8307800292969], dtype: float
Rows: 1
[311.548583984375], dtype: float
Rows: 1
[311.

In [133]:
evalSFrame = gl.SFrame(model_data)
evalSFrame['predicted'] = prediction_list


In [138]:
evalSFrame['per_err'] = evalSFrame['price']/(evalSFrame['predicted'])
evalSFrame['per_err'].max()

RuntimeError: Runtime Exception. Cannot perform on non-numeric types!

 model_train, model_test = model_data.random_split(.75, seed=5)

model_date_train_mask = model_data['int_times']>model_data['int_times'].mean()
model_date_train = model_data[model_date_train_mask]

model_date_test_mask = model_data['int_times']<model_data['int_times'].mean()
model_date_test = model_data[model_date_test_mask]

evalSFrame = gl.SFrame()
evaldateSFrame = gl.SFrame()

model_date = gl.regression.create(model_date_train, target='price')
model = gl.regression.create(model_train, target='price')

price_prediction = model.predict(model_test)
price_date_prediction = model_date.predict(model_date_test)

price_prediction.size()

evalSFrame['real'] = model_test['price']
evalSFrame['predict'] = price_prediction
evalSFrame['per_err'] = evalSFrame['real']/(evalSFrame['predict'])
evalSFrame['per_err'].mean()

evaldateSFrame['real'] = model_date_test['price']
evaldateSFrame['predict'] = price_date_prediction
evaldateSFrame['per_err'] = evaldateSFrame['real']/(evaldateSFrame['predict'])
evaldateSFrame['per_err'].mean()

evaldateSFrame.show(view='ScatterPlot')

evaldateSFrame.head()

evalSFrame['error_per'] = evalSFrame['real'].divide(evalSFrame['predict'])